In [ ]:
# Cálcula la SAM para información en malla de archivos lo suficientemente
# pequeños como para cargarlos por completo en la RAM.

import numpy as np
import pandas as pd
import xarray as xr

In [ ]:
# Cargamos el archivo.
fdir = "../data/CHIRPS_megalopolis.nc"
ds_tot = xr.open_dataset( fdir )

# Ajustamos nombres.
ds_tot = ds_tot.rename_vars( { "longitude": "LONGITUD", "latitude": "LATITUD",
    "time": "TIEMPO", "precip": "PRECIPITACION" } ).swap_dims(
    { "longitude": "LONGITUD", "latitude": "LATITUD", "time": "TIEMPO" } 
    ).to_dataframe().to_xarray()

ds_tot

In [ ]:
# Cálculo de SAM.

pre = "PRECIPITACION"
dur = [1, 2,  3,  4,  5, 10, 15, 20, 30, 40, 50, 60]

# Seleccionamos el área de estudio.
j = 0
ds_i = ds_tot.isel(
    { "LONGITUD": slice( j, ds_tot["LONGITUD"].shape[0] - j ),
       "LATITUD": slice( j, ds_tot["LATITUD" ].shape[0] - j ) } )

# Cantidad de años de estudio.
years = ds_i["TIEMPO"].dt.year.to_dataframe()["year"].unique()
ny = years.shape[0]

# SAM para cada celda.
ds_tret = []
# Iteramos para cada duración.
for i in range( len(dur) ):
    # Creamos un dataset para colocar la SAM.
    ds_y = ds_i.isel( {"TIEMPO": range(0, ny) } ).copy()
    ds_tret.append( ds_y )
    ds_tret[i]["TIEMPO"] = years
    # Agregamos una variable para guardar el año en que se presenta el máximo.
    ds_tret[i]["AÑO"] = ds_tret[i]["PRECIPITACION"].copy()
    ds_p = ds_tret[i]["PRECIPITACION"]
    ds_a = ds_tret[i]["AÑO"]

    # Obtenemos la precipitación acumulada para la duración.
    ds_t = ds_i.rolling( {"TIEMPO": dur[i]} ).sum()
    # Obtenemos el máximo para cada año.
    for y in years:
        ds_p[ ds_p["TIEMPO"] == y ] = ( ds_t[pre].sel(
            { "TIEMPO": (ds_t["TIEMPO"].dt.year == y) }
            ).max(dim = "TIEMPO") )

     # Iteramos para cada celda.
    for lon in ds_tret[i]["LONGITUD"]:
        for lat in ds_tret[i]["LATITUD"]:
            # Ordenamos la SAM.
            df_p = ds_p[ :, ds_p["LONGITUD"] == lon, ds_p["LATITUD"] == lat
                ].to_dataframe().sort_values( pre, ascending = True 
                ).reset_index().rename( {"TIEMPO": "AÑO"}, axis = 1 )
            df_p["TIEMPO"] = years
            df_p = df_p.set_index( ["TIEMPO", "LONGITUD", "LATITUD"] )
            # Asignamos los valores ordenados.
            ds_p[ :, ds_p["LONGITUD"] == lon, ds_p["LATITUD"] == lat ] = (
                df_p.to_xarray()[pre] )
            # Asignamos los años en que se presentan los máximos.
            ds_a[ :, ds_a["LONGITUD"] == lon, ds_a["LATITUD"] == lat ] = (
                df_p.to_xarray()["AÑO"] )
    
    # Calculamos el tiempo de retorno.
    ds_tret[i]["TIEMPO"] = ( ny + 1 ) / np.arange(ny, 0, -1)
    ds_tret[i] = ds_tret[i].rename_dims( {"TIEMPO": "TIEMPO_RETORNO"} 
                          ).rename_vars( {"TIEMPO": "TIEMPO_RETORNO"} )
    
    ds_tret[i] = ds_tret[i].expand_dims( {"DURACION": [dur[i]] } )

ds_trt = xr.concat(ds_tret, "DURACION")

ds_trt